WEB SCRAPPING

In [ ]:
from bing_image_downloader import downloader
from PIL import Image
import os

# Define search terms and parameters
search_terms = ["bank statement", "profit and loss statement", "balance sheet", "payslip"]
output_dir = r"E:\OCR BANK STATMENTS PROJECT\VS CODE INFOSYS\OCR BANK STATMENTS\extracted images"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Download images for each search term
for term in search_terms:
    downloader.download(term, limit=50, # Set number of images per term
                        output_dir=output_dir,
                        adult_filter_off=True, 
                        force_replace=False,
                        timeout=60)

    # Path for images of the current term
    term_dir = os.path.join(output_dir, term)
    
    # Gather all image paths in a list
    image_paths = [os.path.join(term_dir, img) for img in os.listdir(term_dir) if img.endswith(('jpg', 'jpeg', 'png'))]
    
    # Convert images to PDF
    if image_paths:  # Ensure there are images to process
        images = [Image.open(img_path).convert('RGB') for img_path in image_paths]
        
        # Save images as PDF
        pdf_path = os.path.join(output_dir, f"{term.replace(' ', '_')}.pdf")
        images[0].save(pdf_path, save_all=True, append_images=images[1:])
        
print("Images downloaded and saved as PDFs.")


FETCH IMAGES FROM DATA BASE

In [ ]:
import cloudinary
import cloudinary.api
import requests
import os

# Configure Cloudinary with your credentials
cloudinary.config(
    cloud_name='dopwnz1ze',
    api_key='172931444248964',
    api_secret='x6UFGqc1cSfBWcrTAGy7odv8duA'
)

# List of Cloudinary folders to retrieve images from
cloudinary_folders = ["BALANCE SHEET","PROFIT AND LOSS", "BANK STATMENTS", "PAYSLIP"]  # Add your folder names here
output_base_dir = r"E:\OCR BANK STATMENTS PROJECT\VS CODE INFOSYS\OCR BANK STATMENTS\extracted images"  # Base directory to save images

# Create the base output directory if it doesn't exist
os.makedirs(output_base_dir, exist_ok=True)

# Retrieve images from each Cloudinary folder
for folder in cloudinary_folders:
    print(f"Retrieving images from Cloudinary folder: {folder}")
    
    # Create a unique subdirectory for each Cloudinary folder
    folder_output_dir = os.path.join(output_base_dir, folder.replace('/', '_'))
    os.makedirs(folder_output_dir, exist_ok=True)
    
    # Pagination handling for large folders
    next_cursor = None
    while True:
        try:
            # Fetch images from Cloudinary folder (up to 100 per request)
            response = cloudinary.api.resources(type="upload", prefix=folder, max_results=100, next_cursor=next_cursor)
            images = response.get('resources', [])
            next_cursor = response.get('next_cursor')  # For paginated results
            
            # Process each image
            for image in images:
                image_url = image['secure_url']
                image_public_id = image['public_id']
                print(f"Image URL: {image_url}")
                
                # Download and save each image to its folder-specific subdirectory
                img_data = requests.get(image_url).content
                img_filename = os.path.join(folder_output_dir, f"{image_public_id.replace('/', '_')}.jpg")
                with open(img_filename, 'wb') as handler:
                    handler.write(img_data)
                    print(f"Downloaded {img_filename}")
            
            # Break the loop if there are no more pages
            if not next_cursor:
                break
        
        except Exception as e:
            print(f"Error retrieving images from Cloudinary folder '{folder}': {e}")
            break

print("Image retrieval and download completed for all folders.")
